In [83]:
import numpy as np
import pandas as pd
artist_conv ={}
with open("artist_conversions.csv", 'r') as f:
    for line in f:
        l = line.split(',')
        a_id = l[0]
        artist_name = l[1]
        artist_name = artist_name.strip()
        artist_conv[a_id]  = artist_name

In [84]:
from surprise import Dataset
from surprise import Reader

reader = Reader(sep=',', rating_scale=(0,100))
data = Dataset.load_from_file( 'finalUserData.csv', reader=reader)
data.raw_ratings[:5]


[('1', '1', 1.0, None),
 ('2', '1', 61.0, None),
 ('3', '1', 1.0, None),
 ('4', '1', 1.0, None),
 ('5', '1', 101.0, None)]

In [85]:
from surprise import NMF
from surprise.model_selection import train_test_split

trainset, testset = train_test_split( data , test_size= .50 , random_state=1)

algo = NMF( n_factors=10, n_epochs=500, random_state=1 )
algo.fit(trainset)

In [92]:
from surprise import accuracy
pred = algo.test(testset) 
accuracy.rmse(pred), accuracy.mae(pred)
error = "NMF: " + str(accuracy.rmse(pred))+"\n"
with open("RMSE.txt", "a") as w:
    w.write(error)

RMSE: 27.8696
MAE:  16.6633
RMSE: 27.8696


In [87]:

from collections import defaultdict
def get_top_n(predictions, n=3):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n



top_n = get_top_n(pred, n=3)
count = 0
recommendations = []
for uid, user_ratings in top_n.items():
    count+=1
    #print(uid, [iid for (iid, _) in user_ratings])
    ur = [iid for (iid, _) in user_ratings]
    main = str(uid)+","+str(artist_conv[str(ur[0])])+ ","+str(artist_conv[str(ur[1])])+","+str(artist_conv[str(ur[2])])+"\n"
    print(main)
    recommendations.append(main)
print(count)

415,The-Clash,Queens-Of-The-Stone-Age,Flogging-Molly

577,Pink-Floyd,The-Beatles,Led-Zeppelin

277,Pearl-Jam,Rage-Against-The-Machine,Tool

61,Evanescence,Coldplay,Barenaked-Ladies

194,Primus,Jimi-Hendrix-Experience,Pink-Floyd

198,Marvin-Gaye,Billy-Joel,Genesis

428,Audioslave,Beck,Rage-Against-The-Machine

299,Van-Halen,Led-Zeppelin,AC/DC

539,Marvin-Gaye,Dido,Luther-Vandross

112,The-Beatles,Creedence-Clearwater-Revival,Lynyrd-Skynyrd

174,Outkast,Nirvana,Rage-Against-The-Machine

53,Veruca-Salt,Linkin-Park,Faith-No-More

127,Elton-John,Aerosmith,Toad-The-Wet-Sprocket

161,Counting-Crows,Garbage,Evanescence

510,Led-Zeppelin,Pink-Floyd,The-Police

460,Aerosmith,Journey,Poison

232,Anita-Baker,The-Monkees,Janet-Jackson

23,Pink-Floyd,Nirvana,Led-Zeppelin

512,R.E.M.,Smashing-Pumpkins,Garbage

22,Linkin-Park,Godsmack,Matchbox-Twenty

575,Outkast,Eminem,LL-Cool-J

342,Eminem,Outkast,Destiny's-Child

178,Staind,Incubus,Godsmack

117,Foo-Fighters,Live,Weezer

199,Billy-Joel,Eagles,Tom-P

In [91]:
with open("NMF.txt", "w") as w:
    for x in recommendations:
        w.write(x)